# 1. Basics

- [torch官方](https://github.com/pytorch/examples/tree/main/distributed/ddp-tutorial-series)


1. multiple GPUs in a single machine/server/node: 单机多卡

    - 分布式数据并行时，模型(model parameters)/ 优化器(optimizer states)每张卡都会拷贝一份(replicas)
        - DDP始终在卡间维持着模型参数和优化器状态的同步一致性在整个训练过程中
    
    - DataParallel, batch input通过DistributedSampler split分发到不同的GPUs上
        - 模型/optimizer相同，但因为输入数据不同，loss不同，反向传播时计算的梯度也不同
        - 此时DDP如何保证卡间的同步一致性
            - ring all-reduce algorithm


2. ring all-reduce algorithm

    - 将所有的gpus连成一个ring
    - 同步过程，不需要等待所有的卡都计算完一轮梯度
    - 经过这个同步过程，所有的卡的model/optimizer都会保持一致状态


## 1.1 Ring All-Reduce Algorithm

1. 简介
    - 李沐：参数服务器
    - 计算和同步的几个过程
        - GPUs分别计算损失（forward），梯度（backward）
        - 梯度聚合
        - 模型/优化器参数的更新以及广播

2. HPC的基础算法

3. Ring环形拓扑结构
    - 环形，逻辑的
    - 两个过程：
        - scatter-reduce
        - all gather

# 2. DDP相关概念

## 2.1 node，rank，world_size

1. world，world_size
    - world: 一个组group，包含了一组分布式训练的进程
        - 通常，每一个gpu代表一个进程（process）
        - world内的process可以彼此通信，所以有DDP分布式训练

2. rank
    - rank：进程的唯一标识，进程级别的概念，识别进程，因为进程之间需要通信
    - local rank：多机多卡概念

3. node
    - 理解为一个server机器，2个servers就是2个nodes

> 例如2个servers（node），各有四张卡gpus
> world_size:2*4==8, ranks:[0,1,2,3,4,5,6,7], local_rank:[0,1,2,3], [0,1,2,3]